In [ ]:
!pip install transformers
!telegram.txt
!pip install --upgrade diffusers
!pip install transformers
!pip install accelerate
!pip install transformers torch pillow
!pip install openai
!pip install python-telegram-bot
!pip install youtube_transcript_api
!pip install openai
!pip install PyPDF2
!pip install --upgrade python-telegram-bot
!pip install python-telegram-bot==13.7
!pip install pyTelegramBotAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.3 MB/s eta 0:00:00
/bin/bash: telegram.txt: command not found
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 852.0/852.0 kB 7.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 15.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-pytho

In [ ]:
import io
import telegram
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image
import os
from diffusers import DiffusionPipeline
from diffusers import StableDiffusionPipeline

# Define the token for your bot obtained from BotFather
TOKEN = '6192991239:AAG8aIa870XthmJUO5ovEr6gvVk6kTht88k'

# Create the necessary objects for image captioning
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
model_id = "prompthero/openjourney"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
print("model loaded")
# Define the function for handling the "/start" command
def start(update, context):
    update.message.reply_text('Hello! Send me an image to get a caption.')

# Define the function for handling image messages
def handle_image(update, context):
    # Get the file ID and download the image
    file_id = update.message.photo[-1].file_id
    file = context.bot.getFile(file_id)
    filename = os.path.join(os.getcwd(), f"{file_id}.jpg")
    file.download(filename)

    # Run inference on the image and return the caption
    caption = predict_step([filename])[0]
    update.message.reply_text(f"Caption: {caption}")

    # Delete the downloaded image file to free up space
    os.remove(filename)

# Define the function for running inference on an image
def predict_step(image_paths):
    images = []
    for image_path in image_paths:
        i_image = Image.open(image_path)
        if i_image.mode != "RGB":
            i_image = i_image.convert(mode="RGB")
        images.append(i_image)

    pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    output_ids = model.generate(pixel_values, **gen_kwargs)

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]
    return preds

# Define the function for handling prompt messages
def handle_prompt(update, context):
    # Get the prompt message
    prompt = update.message.text

    # Use diffusion to generate an image based on the prompt
    image = generate_image(prompt)
    print(image)
    image_byte_array = io.BytesIO()
    image.save(image_byte_array, format='JPEG')
    image_byte_array.seek(0)

    # Send the generated image to the user
    context.bot.send_photo(chat_id=update.message.chat_id, photo=image_byte_array)

    # Send the generated image to the user
    # context.bot.send_photo(chat_id=update.message.chat_id, photo=image)

# Define the function for generating an image based on a prompt
def generate_image(prompt):
    print("helo")

    image = pipe(prompt).images[0]
    return image

# Create the Updater and add the handlers
updater = Updater(TOKEN, use_context=True)
dp = updater.dispatcher
dp.add_handler(CommandHandler("start", start))
dp.add_handler(MessageHandler(Filters.photo, handle_image))
dp.add_handler(MessageHandler(Filters.text, handle_prompt))

# Start the bot
updater.start_polling() 
updater.idle()
